In [2]:
import itertools
import math
import numpy as np
import pandas as pd
from scipy import stats
import sys

Generate sequences and their corresponding REXL definitions

In [2]:
np.random.seed(12345)
np.set_printoptions(
    precision=None,
    nanstr='0.0/0.0',
    infstr='1.0/0.0',
    threshold=sys.maxsize,
    linewidth=sys.maxsize,
    floatmode='maxprec',
    suppress=True)

A0_1 = (np.random.normal(0, 1, 30), "A0_1")
B0_1 = (np.random.normal(0, 1, 30), "B0_1")
A0_5 = (np.random.normal(0, 5, 30), "A0_5")
A1_1 = (np.random.normal(1, 1, 30), "A1_1")
N0_1 = (np.concatenate(([math.nan], A0_1[0])), "N0_1")

samps = [A0_1, B0_1, A0_5, A1_1, N0_1]

for x, name in samps:
    print(f"`` {name} := {np.array2string(x, separator=', ')};")

`` A0_1 := [-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057,  1.39340583,  0.09290788,  0.28174615,  0.76902257,  1.24643474,  1.00718936, -1.29622111,  0.27499163,  0.22891288,  1.35291684,  0.88642934, -2.00163731, -0.37184254,  1.66902531, -0.43856974, -0.53974145,  0.47698501,  3.24894392, -1.02122752, -0.5770873 ,  0.12412128,  0.30261356,  0.52377207,  0.00094028,  1.34380979];
`` B0_1 := [-0.71354399, -0.83115354, -2.37023165, -1.86076079, -0.8607574 ,  0.56014529, -1.26593449,  0.11982712, -1.06351245,  0.33288272, -2.35941881, -0.19954296, -1.54199553, -0.97073591, -1.30703025,  0.28634975,  0.37798411, -0.75388653,  0.33128565,  1.34974221,  0.06987669,  0.24667411, -0.0118616 ,  1.00481159,  1.32719461, -0.91926156, -1.54910644,  0.0221846 ,  0.75836315, -0.66052433];
`` A0_5 := [  4.31290041,  -0.05015951,   0.25004678,   3.35107797,   4.26482516,  -4.77934426,  -0.1174666 , -11.52116939,  -3.26234421,  -6.09150988,  -6.66304854,   5.37311345,   3.61820753, 

Sequence definitions

In [3]:
def fmt_seq(seq_items):
    return '[ ' + ', '.join(str(s) for s in seq_items) + ' ]'

def fmt_tup_w_name(samp):
    return f'({samp[1]}, "{samp[1]}")'

# Include null sequences for the REXL definitions, but not for use in the Python code.
null_pair = ("null", "null")
null_pairs = []
for i in range(3):
    a = samps[0] if i & 1 != 0 else null_pair
    b = samps[0] if i & 2 != 0 else null_pair
    null_pairs.append((a, b))

samps_def = fmt_seq(fmt_tup_w_name(s) for s in samps + [null_pair])
print(f"`` Samps := {samps_def};")

pairs = list(itertools.combinations(samps, 2))

pairs_def = fmt_seq([f'({fmt_tup_w_name(p[0])}, {fmt_tup_w_name(p[1])})' for p in pairs + null_pairs ])
print(f"`` Pairs := {pairs_def};")

`` Samps := [ (A0_1, "A0_1"), (B0_1, "B0_1"), (A0_5, "A0_5"), (A1_1, "A1_1"), (N0_1, "N0_1"), (null, "null") ];
`` Pairs := [ ((A0_1, "A0_1"), (B0_1, "B0_1")), ((A0_1, "A0_1"), (A0_5, "A0_5")), ((A0_1, "A0_1"), (A1_1, "A1_1")), ((A0_1, "A0_1"), (N0_1, "N0_1")), ((B0_1, "B0_1"), (A0_5, "A0_5")), ((B0_1, "B0_1"), (A1_1, "A1_1")), ((B0_1, "B0_1"), (N0_1, "N0_1")), ((A0_5, "A0_5"), (A1_1, "A1_1")), ((A0_5, "A0_5"), (N0_1, "N0_1")), ((A1_1, "A1_1"), (N0_1, "N0_1")), ((null, "null"), (null, "null")), ((A0_1, "A0_1"), (null, "null")), ((null, "null"), (A0_1, "A0_1")) ];


Boilerplate

In [4]:
alts = ["two-sided", "greater", "less"]

pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
pd.options.display.width = 1000
pd.options.display.float_format = lambda val: f'{val:.15g}'

Generate one-sample results

In [5]:
one_table = []
for x, name in samps:
    for mean in [-1, 0, 1]:
        
        res = {
            "X": name,
            "Popmean": mean
        }

        for alt in alts:
            tres = stats.ttest_1samp(x, mean, alternative=alt)
            res[alt.capitalize()] = tres.pvalue
            res["T"] = tres.statistic
            res["Dof"] = tres.df

        one_table.append(res)

Generate two-sample results

In [6]:
ind_table = []
for p in pairs:
    x, y = [s[0] for s in p]
    xname, yname = [s[1] for s in p]
    for eqv in [False, True]:
        
        res = {
            "X": xname,
            "Y": yname,
            "EqualVar": eqv
        }

        for alt in alts:
            tres = stats.ttest_ind(x, y, equal_var=eqv, alternative=alt)
            res[alt.capitalize()] = tres.pvalue
            res["T"] = tres.statistic

        ind_table.append(res)

Generate paired results

In [7]:
paired_table = []
for p in pairs:
    x, y = [s[0] for s in p]
    xname, yname = [s[1] for s in p]

    res = {
        "X": xname,
        "Y": yname,
    }

    for alt in alts:
        tres = stats.ttest_rel(x, y, alternative=alt)
        res[alt.capitalize()] = tres.pvalue
        res["T"] = tres.statistic
        res["Dof"] = tres.df
    
    paired_table.append(res)

# One-Sample t-test Results

In [8]:
one_frame = pd.DataFrame(one_table).reindex(columns=["X", "Popmean", "Dof", "Two-sided", "Less", "Greater", "T"])
print(one_frame)

       X  Popmean  Dof            Two-sided                 Less              Greater                  T
0   A0_1       -1   29 1.37241229399909e-07    0.999999931379385 6.86206146999544e-08     6.906285439073
1   A0_1        0   29   0.0915780132254632    0.954210993387268   0.0457890066127316   1.74498284814515
2   A0_1        1   29  0.00189799253299714  0.00094899626649857    0.999051003733501   -3.4163197427827
3   B0_1       -1   29  0.00351283226531763    0.998243583867341  0.00175641613265882   3.17768752986157
4   B0_1        0   29   0.0318643803064591   0.0159321901532296     0.98406780984677  -2.25485643836581
5   B0_1        1   29 1.77809826855226e-08 8.89049134276131e-09    0.999999991109509  -7.68740040659319
6   A0_5       -1   29    0.222076812290201      0.8889615938549      0.1110384061451   1.24782613510751
7   A0_5        0   29    0.925286930184926    0.537356534907537    0.462643465092463 0.0945952332178155
8   A0_5        1   29    0.298504309864489    0.149252

# Two-Sample t-test Results

In [9]:
ind_frame = pd.DataFrame(ind_table).reindex(columns=["X", "Y", "EqualVar", "Two-sided", "Less", "Greater", "T"])
print(ind_frame)

       X     Y  EqualVar            Two-sided                 Less             Greater                  T
0   A0_1  B0_1      True    0.006594935747841     0.99670253212608  0.0032974678739205   2.81815889328936
1   A0_1  B0_1     False  0.00659989372740455    0.996700053136298 0.00329994686370227   2.81815889328936
2   A0_1  A0_5      True    0.774225772900765    0.612887113549618   0.387112886450382  0.288193814013827
3   A0_1  A0_5     False    0.775063334802974    0.612468332598513   0.387531667401487  0.288193814013827
4   A0_1  A1_1      True  0.00870644390777613  0.00435322195388807   0.995646778046112  -2.71541161790461
5   A0_1  A1_1     False  0.00872328753865265  0.00436164376932633   0.995638356230674  -2.71541161790461
6   A0_1  N0_1      True                  NaN                  NaN                 NaN                NaN
7   A0_1  N0_1     False                  NaN                  NaN                 NaN                NaN
8   B0_1  A0_5      True    0.577117950744803 

# Paired t-test Results

In [10]:
paired_frame = pd.DataFrame(paired_table).reindex(columns=["X", "Y", "Dof", "Two-sided", "Less", "Greater", "T"])
print(paired_frame)

      X     Y  Dof            Two-sided                 Less             Greater                  T
0  A0_1  B0_1   29   0.0118895217898545    0.994055239105073 0.00594476089492724   2.68422691922095
1  A0_1  A0_5   29    0.777780971172766    0.611109514413617   0.388890485586383  0.284853246006453
2  A0_1  A1_1   29   0.0129506424264915  0.00647532121324574   0.993524678786754  -2.64827613986959
3  A0_1  N0_1  NaN                  NaN                  NaN                 NaN                NaN
4  B0_1  A0_5   29    0.584986582788624    0.292493291394312   0.707506708605688 -0.552282522403587
5  B0_1  A1_1   29 2.78179975094044e-06 1.39089987547022e-06   0.999998609100125  -5.79725300783954
6  B0_1  N0_1  NaN                  NaN                  NaN                 NaN                NaN
7  A0_5  A1_1   29    0.291848671138791    0.145924335569396   0.854075664430604  -1.07361320791624
8  A0_5  N0_1  NaN                  NaN                  NaN                 NaN                NaN
